In [5]:
import os
import torch

file_path = "server_state.pt"
saved_state = torch.load(file_path) 
cos_matrices = saved_state['cos_matrices']

In [2]:
print(len(cos_matrices))

1


In [4]:
print(cos_matrices[0])

[[ 1.          0.73643869  0.17096032  0.16592796  0.16737138  0.24723968
   0.18336922  0.29147723  0.29233855  0.40233177]
 [ 0.73643869  1.          0.1761723   0.16600716  0.12457844  0.24931687
   0.18331745  0.3016524   0.30733451  0.39013758]
 [ 0.17096032  0.1761723   1.         -0.01869179  0.00445957  0.02323162
   0.01563538  0.14882725  0.07518217  0.14629312]
 [ 0.16592796  0.16600716 -0.01869179  1.         -0.05309801  0.02030551
   0.00840923  0.02955849  0.51656455 -0.00157906]
 [ 0.16737138  0.12457844  0.00445957 -0.05309801  1.         -0.00510852
   0.02446726  0.12780051 -0.00979686  0.18863305]
 [ 0.24723968  0.24931687  0.02323162  0.02030551 -0.00510852  1.
   0.01399827  0.46875218  0.05236563  0.27886075]
 [ 0.18336922  0.18331745  0.01563538  0.00840923  0.02446726  0.01399827
   1.          0.01025898  0.1687959   0.23141254]
 [ 0.29147723  0.3016524   0.14882725  0.02955849  0.12780051  0.46875218
   0.01025898  1.          0.04307675  0.12773371]
 [ 0.292

In [1]:
import numpy as np

# 生成一個 10x10 的隨機數組
array = np.random.rand(10, 10)

# 打印結果
print(array)

[[0.62755407 0.98220582 0.409962   0.9782935  0.17301276 0.10001542
  0.0123776  0.33725442 0.58757667 0.13459402]
 [0.90243721 0.73061109 0.78814509 0.88283126 0.97217616 0.21379501
  0.80303417 0.49388237 0.13773149 0.22666628]
 [0.98794962 0.36299246 0.13196574 0.75760467 0.30888315 0.82752005
  0.76548993 0.39730877 0.78767993 0.78615258]
 [0.21222417 0.32907782 0.59323487 0.5957187  0.49508776 0.67954313
  0.80477312 0.57145753 0.32170959 0.31129393]
 [0.39837834 0.33822137 0.29441373 0.33300694 0.73404597 0.98173653
  0.95475967 0.47410045 0.16231077 0.34507192]
 [0.72241385 0.40199144 0.46806866 0.93478094 0.95219032 0.11494384
  0.54413304 0.01596938 0.88097639 0.49260855]
 [0.98292666 0.23004204 0.37957762 0.8370672  0.13135696 0.48807521
  0.61628462 0.98994645 0.89799716 0.93841171]
 [0.62623243 0.31273445 0.16227849 0.23671501 0.38607771 0.39128803
  0.5260048  0.7153111  0.50958152 0.91943246]
 [0.8945418  0.38390027 0.95760272 0.44256345 0.28516229 0.36808108
  0.86811483

In [2]:
import numpy as np
import cvxpy as cp

def optimizing_graph_matrix_neighbor(model_cosine_matrix, alpha, fed_avg_freqs):
    n = model_cosine_matrix.shape[0]
    graph_matrix = np.zeros((n, n))
    p = np.array(list(fed_avg_freqs.values()))
    P = alpha * np.identity(n)
    # P = cp.atoms.affine.wraps.psd_wrap(P)
    G = - np.identity(n)
    h = np.zeros(n)
    A = np.ones((1, n))
    b = np.ones(1)
    for i in range(n):
        model_cosine_vector = model_cosine_matrix[i]
        d = model_cosine_vector
        q = d - 2 * alpha * p
        x = cp.Variable(n)
        prob = cp.Problem(cp.Minimize(cp.quad_form(x, P) + q.T @ x),
                  [G @ x <= h,
                   A @ x == b]
                  )
        prob.solve()

        graph_matrix[i, :] = torch.Tensor(x.value)
    return graph_matrix


In [3]:
fed_avg_freqs = {key: 0.1 for key in range(10)}
alpha = 0.8 # paper 跟 code中的alpha, lambda剛好是反過來的!

In [8]:
updated_graph_matrix = optimizing_graph_matrix_neighbor(-array, alpha, fed_avg_freqs)

In [10]:
print(updated_graph_matrix[0,:])

[1.44022286e-01 3.65679622e-01 8.02724250e-03 3.63234431e-01
 2.04270356e-23 2.04269473e-23 4.23897689e-23 1.49364139e-23
 1.19036414e-01 4.23898226e-23]


In [12]:
print(updated_graph_matrix[7,:])

[ 1.04830123e-01  1.11189604e-01  1.56738833e-02 -1.05091647e-23
  2.53216806e-03  2.15626955e-01 -2.14905669e-23  5.47656834e-01
 -3.24719666e-23  2.49041687e-03]


In [22]:
print(cos_matrices[0][1])

[0.73643869 1.         0.1761723  0.16600716 0.12457844 0.24931687
 0.18331745 0.3016524  0.30733451 0.39013758]
